In [1]:
import pandas as pd
import numpy as np

In [89]:
# Save DataFrame to CSV without indexes
df2.to_csv('HR_cleaned_data.csv', index=False)

## dump the data in mysql

In [1]:
!pip install pandas mysql-connector-python

In [2]:
import pandas as pd
import mysql.connector
from mysql.connector import Error


In [48]:
df = pd.read_csv('movies_revenue.csv')


In [49]:
df

,movie_id,revenue,budget,release_year
0,1,825532764,160000000,2010
1,2,289327971,25000000,1994
2,3,213928762,8000000,1994
3,4,1004558444,185000000,2008
4,5,678226247,55000000,1994
5,6,1118888979,94000000,2003
6,7,245066411,6000000,1972
7,8,465343787,63000000,1999
8,9,100853753,63000000,1999
9,10,46628752,25000000,1990


In [50]:
df.dtypes

movie_id        int64
revenue         int64
budget          int64
release_year    int64
dtype: object

In [11]:
df.drop(columns= ['Unnamed: 3',"Unnamed: 4"],inplace=True)
df

,eid,Job,Salary
0,1.0,Mangaer,20000.0
1,2.0,Clerk,42000.0
2,3.0,Salesman,13000.0
3,4.0,Clerk,7000.0
4,5.0,Salesman,13000.0
...,...,...,...
541668,NaN,NaN,NaN
541669,NaN,NaN,NaN
541670,NaN,NaN,NaN
541671,NaN,NaN,NaN


In [12]:
df.dropna(how='all',inplace=True)
df

,eid,Job,Salary
0,1.0,Mangaer,20000.0
1,2.0,Clerk,42000.0
2,3.0,Salesman,13000.0
3,4.0,Clerk,7000.0
4,5.0,Salesman,13000.0
5,6.0,Clerk,22000.0


In [13]:
# Save DataFrame to CSV without indexes
df.to_csv('cleaned_data.csv', index=False)

In [51]:
try:
    connection = mysql.connector.connect(host='localhost',
                                         database='testdb',
                                         user='root',
                                         password='Root@#123$')
    if connection.is_connected():
        db_Info = connection.get_server_info()
        print("Connected to MySQL Server version ", db_Info)
except Error as e:
    print("Error while connecting to MySQL", e)


Connected to MySQL Server version  8.0.28


In [52]:
# Define a mapping between pandas data types and MySQL data types
data_types_mapping = {
    'int64': 'INT',
    'float64': 'FLOAT',
    'object': 'TEXT',  # For simplicity, assuming all non-numeric types are stored as text
    'datetime64': 'DATETIME'
}

In [53]:
# Create Table
cursor = connection.cursor()

# Get column names and their data types from DataFrame
columns = ",".join([f"{col} {data_types_mapping[str(df[col].dtype)]}" for col in df.columns])

# Create table with columns
cursor.execute(f"CREATE TABLE IF NOT EXISTS revenue ({columns})")

# Insert Data
# Convert DataFrame values to Python built-in types
data = [[int(x) if isinstance(x, np.int64) else x for x in row] for row in df.values]
cursor.executemany(f"INSERT INTO revenue VALUES ({','.join(['%s'] * len(df.columns))})", data)

# Commit Changes and Close Connection
connection.commit()
cursor.close()
connection.close()
